In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Tuple
import os
import re

In [2]:
# translate column of secondary reason for override

In [3]:
overrides = pd.read_excel(r"C:\Users\n740789\Documents\Projects_local\DataSets\overrides\202503_BBDD_Overrides.xlsx",
                   sheet_name="MAPEO P-S",
                   dtype={'permId (Clarity)':str, 'ClarityID':str},
                   )

In [4]:
overrides.columns

Index(['#', 'ClarityID', 'permId (Clarity)', 'IssuerID', 'Issuer Name',
       'SNT-WORLD\n(Y/-)', 'PARENT NAME (Aladdin)',
       'PARENT IssuerID (Aladdin)', 'Parent (P) / Subsidiaria (S)',
       'PARENT NAME (Clarity)', 'Parent ClarityID', 'GICS_2', 'GICS_2\nParent',
       'Sustainability Rating Parent', 'OVR Inheritance BiC',
       'OVR STR001\n(SEC)', 'OVR STR002\n(EC*)', 'OVR STR003\n(EC)',
       'OVR STR003B\n(EC*)', 'OVR STR004\n(SEC)', 'OVR STR005\n(EC*)',
       'OVR STR006\n(SEC)', 'OVR STR007\n(SEC)', 'OVR CS001\n(SEC*)',
       'OVR CS002\n(EC*)', 'OVR CS003\n(SEC)', 'OVR ARTICULO 8\n(C)',
       'Motivo principal OVR', 'Motivo secundario OVR',
       'Detalle OVR/Observación adicional', 'Fecha aplicación OVR'],
      dtype='object')

In [5]:
overrides.columns = (
    overrides.columns.str.strip()
    .str.replace(' ', '_')
    .str.replace(")","")\
    .str.replace("(","")
    .str.replace("/","")
    .str.lower()
    .str.replace("\n","_")
    .str.replace("*","")
    .str.replace("__","_")
    )

In [6]:
overrides.columns

Index(['#', 'clarityid', 'permid_clarity', 'issuerid', 'issuer_name',
       'snt-world_y-', 'parent_name_aladdin', 'parent_issuerid_aladdin',
       'parent_p_subsidiaria_s', 'parent_name_clarity', 'parent_clarityid',
       'gics_2', 'gics_2_parent', 'sustainability_rating_parent',
       'ovr_inheritance_bic', 'ovr_str001_sec', 'ovr_str002_ec',
       'ovr_str003_ec', 'ovr_str003b_ec', 'ovr_str004_sec', 'ovr_str005_ec',
       'ovr_str006_sec', 'ovr_str007_sec', 'ovr_cs001_sec', 'ovr_cs002_ec',
       'ovr_cs003_sec', 'ovr_articulo_8_c', 'motivo_principal_ovr',
       'motivo_secundario_ovr', 'detalle_ovrobservación_adicional',
       'fecha_aplicación_ovr'],
      dtype='object')

In [7]:
ovr_filtered = overrides[
    [
        "clarityid", 
        "permid_clarity",
        "issuerid",
        "issuer_name",
        "parent_p_subsidiaria_s",
        "parent_name_clarity",
        "parent_clarityid",
        "parent_name_aladdin",
        "parent_issuerid_aladdin",
        "ovr_str001_sec",
        "ovr_str002_ec",
        "ovr_str003_ec",
        "ovr_str003b_ec",
        "ovr_str004_sec",
        "ovr_str005_ec",
        "ovr_str006_sec",
        "ovr_articulo_8_c",
        "ovr_cs001_sec",
        "ovr_cs002_ec",
        "ovr_cs003_sec",
        'motivo_principal_ovr',
        'motivo_secundario_ovr'

    ]
].copy()

In [8]:
ovr_filtered.rename(
    columns={
        "parent_p_subsidiaria_s":"parent_sub",
        "issuerid":"brs_id",
        "permid_clarity":"permid",
        "parent_name_aladdin" : "parent_name_brs",
        "parent_issuerid_aladdin" : "parent_brs_id",
        "ovr_str001_sec" : "str_001",
        "ovr_str002_ec" : "str_002",
        "ovr_str003_ec" : "str_003",
        "ovr_str003b_ec" : "str_003b",
        "ovr_str004_sec" : "str_004",
        "ovr_str005_ec" : "str_005",
        "ovr_str006_sec" : "str_006",
        "ovr_articulo_8_c" : "art_8", 
        "ovr_cs001_sec" : "cs_001",
        "ovr_cs002_ec" : "cs_003",
        "ovr_cs003_sec" : "cs_002"
        }, 
        inplace=True
)

In [9]:
#get crossreference
crossreference = pd.read_csv(r"C:\Users\n740789\Documents\Projects_local\DataSets\crossreference\Aladdin_Clarity_Issuers_20250301.csv", 
                             dtype={"CLARITY_AI": str})
crossreference.columns = ["brs_id", "brs_issuer_name", "permid", "msci_id", "sustainalytics_id"]
# drop from crossreference columns msci_id and susctainalytics_id
crossreference.drop(columns=["msci_id", "sustainalytics_id"], inplace=True)

In [10]:
crossreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134952 entries, 0 to 134951
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   brs_id           134952 non-null  object
 1   brs_issuer_name  134952 non-null  object
 2   permid           131894 non-null  object
dtypes: object(3)
memory usage: 3.1+ MB


In [11]:
# read latest datafeed at the issuer level
target_columns = [
    "issuer_name",
    "permid",
    "clarityid",
    "company_inheriting",
    "parent_company",
    "str_001_s",
    "str_002_ec",
    "str_003_ec",
    "str_003b_ec",
    "str_004_asec",
    "str_005_ec",
    "cs_001_sec",
    "cs_003_sec",
    "cs_002_ec",
    "str_006_sec",
    "art_8_basicos"
]

datafeed = pd.read_csv(
    r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\ficheros_tratados\2025\20250401_Equities_feed_IssuerLevel_sinOVR.csv",
    usecols=target_columns,
    dtype={"permid": str, "clarityid": str}
)

In [12]:
# find issuer names in datafeed that contain the word sncf
datafeed[
    datafeed["issuer_name"].str.contains("sncf re", case=False)
]

,issuer_name,company_inheriting,parent_company,str_001_s,str_002_ec,str_003_ec,str_004_asec,clarityid,str_005_ec,cs_001_sec,cs_003_sec,cs_002_ec,str_006_sec,art_8_basicos,permid,str_003b_ec
1172,Sncf Reseau SA,False,-,OK,OK,OK,EXCLUDED,182012,OK,EXCLUDED,EXCLUDED,OK,OK,OK,4295867872,OK


69278, 69264, 69259

In [13]:
datafeed.rename(
    columns={
        "str_001_s"     :   "str_001",
        "str_002_ec"    :   "str_002",
        "str_003_ec"    :   "str_003",
        "str_003b_ec"   :   "str_003b",
        "str_004_asec"  :   "str_004",
        "str_005_ec"    :   "str_005",
        "str_006_sec"   :   "str_006",
        "cs_001_sec"    :   "cs_001",
        "cs_003_sec"    :   "cs_003",
        "cs_002_ec"     :   "cs_002", 
        "art_8_basicos" :   "art_8"    
    },
    inplace=True
)

In [14]:
ovr_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7760 entries, 0 to 7759
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   clarityid              7759 non-null   object
 1   permid                 7759 non-null   object
 2   brs_id                 2518 non-null   object
 3   issuer_name            7759 non-null   object
 4   parent_sub             7759 non-null   object
 5   parent_name_clarity    7757 non-null   object
 6   parent_clarityid       7759 non-null   object
 7   parent_name_brs        7688 non-null   object
 8   parent_brs_id          7685 non-null   object
 9   str_001                7119 non-null   object
 10  str_002                7133 non-null   object
 11  str_003                7116 non-null   object
 12  str_003b               7096 non-null   object
 13  str_004                7124 non-null   object
 14  str_005                7050 non-null   object
 15  str_006              

In [15]:
crossreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134952 entries, 0 to 134951
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   brs_id           134952 non-null  object
 1   brs_issuer_name  134952 non-null  object
 2   permid           131894 non-null  object
dtypes: object(3)
memory usage: 3.1+ MB


In [16]:
# Create a mapping dictionary from permid to brs_id in the crossreference dataframe
mapping = crossreference.set_index('permid')['brs_id'].to_dict()

# Use the mapping to fill missing brs_id values in ovr_filtered based on the permid column
ovr_filtered['brs_id'] = ovr_filtered['brs_id'].fillna(ovr_filtered['permid'].map(mapping))

In [17]:
for c in ovr_filtered.columns:
    print(c)

clarityid
permid
brs_id
issuer_name
parent_sub
parent_name_clarity
parent_clarityid
parent_name_brs
parent_brs_id
str_001
str_002
str_003
str_003b
str_004
str_005
str_006
art_8
cs_001
cs_003
cs_002
motivo_principal_ovr
motivo_secundario_ovr


In [18]:
# List of columns to check
columns_to_check = [
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8", 
    "cs_001",
    "cs_003",
    "cs_002"
]

# Filter rows where all specified columns are empty or NaN
ovr_clean = ovr_filtered[~ovr_filtered[columns_to_check].apply(lambda row: row.isna().all() or (row == '').all(), axis=1)]

In [19]:
ovr_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7201 entries, 1 to 7758
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   clarityid              7201 non-null   object
 1   permid                 7201 non-null   object
 2   brs_id                 2731 non-null   object
 3   issuer_name            7201 non-null   object
 4   parent_sub             7201 non-null   object
 5   parent_name_clarity    7200 non-null   object
 6   parent_clarityid       7201 non-null   object
 7   parent_name_brs        7163 non-null   object
 8   parent_brs_id          7160 non-null   object
 9   str_001                7119 non-null   object
 10  str_002                7133 non-null   object
 11  str_003                7116 non-null   object
 12  str_003b               7096 non-null   object
 13  str_004                7124 non-null   object
 14  str_005                7050 non-null   object
 15  str_006                711

In [20]:
df_target_col =[
    "permid",
    "company_inheriting", 
    "parent_company", 
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8", 
    "cs_001",
    "cs_003",
    "cs_002"
]

In [21]:
ovr_merged = pd.merge(ovr_clean, datafeed[df_target_col], how='left', on='permid', suffixes=('_ovr', '_df'))

In [22]:
ovr_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7201 entries, 0 to 7200
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   clarityid              7201 non-null   object
 1   permid                 7201 non-null   object
 2   brs_id                 2731 non-null   object
 3   issuer_name            7201 non-null   object
 4   parent_sub             7201 non-null   object
 5   parent_name_clarity    7200 non-null   object
 6   parent_clarityid       7201 non-null   object
 7   parent_name_brs        7163 non-null   object
 8   parent_brs_id          7160 non-null   object
 9   str_001_ovr            7119 non-null   object
 10  str_002_ovr            7133 non-null   object
 11  str_003_ovr            7116 non-null   object
 12  str_003b_ovr           7096 non-null   object
 13  str_004_ovr            7124 non-null   object
 14  str_005_ovr            7050 non-null   object
 15  str_006_ovr          

In [23]:
for c in sorted(ovr_merged.columns):
    print(c)

art_8_df
art_8_ovr
brs_id
clarityid
company_inheriting
cs_001_df
cs_001_ovr
cs_002_df
cs_002_ovr
cs_003_df
cs_003_ovr
issuer_name
motivo_principal_ovr
motivo_secundario_ovr
parent_brs_id
parent_clarityid
parent_company
parent_name_brs
parent_name_clarity
parent_sub
permid
str_001_df
str_001_ovr
str_002_df
str_002_ovr
str_003_df
str_003_ovr
str_003b_df
str_003b_ovr
str_004_df
str_004_ovr
str_005_df
str_005_ovr
str_006_df
str_006_ovr


In [24]:
def active_ovr(row, column_pairs):
    result = {}
    for ovr_col, base_col in column_pairs:
        if pd.isna(row[ovr_col]) or pd.isna(row[base_col]):
            result[f"{ovr_col}_active"] = np.nan
        else:
            result[f"{ovr_col}_active"] = row[ovr_col] != row[base_col]
    return result

In [25]:
# Define column pairs to compare based on the provided columns
column_pairs_to_compare = [
    ("str_001_ovr", "str_001_df"),
    ("str_002_ovr", "str_002_df"),
    ("str_003_ovr", "str_003_df"),
    ("str_003b_ovr", "str_003b_df"),
    ("str_004_ovr", "str_004_df"),
    ("str_005_ovr", "str_005_df"),
    ("str_006_ovr", "str_006_df"),
    ("art_8_ovr", "art_8_df"),
    ("cs_001_ovr", "cs_001_df"),
    ("cs_002_ovr", "cs_002_df"),
    ("cs_003_ovr", "cs_003_df")
]

In [26]:

# Add new columns for each override
for index, row in ovr_merged.iterrows():
    active_overrides = active_ovr(row, column_pairs_to_compare)
    for col, value in active_overrides.items():
        ovr_merged.at[index, col] = value

In [27]:
# Fix override grounds
def fix_spell(ground: str) -> str:
    if ground in ['BIC', 'BiC']:
        return 'SR'
    elif ground == 'Exposición':
        return 'exposure'
    elif ground in ['Inheritance', 'Inheritance de subsidiaria', 'Inheritance por caida de SR']:
        return 'inheritance'
    elif ground == 'Comité':
        return 'committee'
    elif ground in ['Controversia', 'Fallo controversias Clarity', 'OVR por controversia nivel 3/5 en Susta pero en Clarity 1/100']:
        return 'controversy'
    else:
        return ground  # Return the original input if no conditions match

ovr_merged.motivo_principal_ovr = overrides.motivo_principal_ovr.apply(fix_spell)

In [28]:
result_df = ovr_merged.copy()

In [29]:
"""
add these columns to result_df:     
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance"
"""
result_df["ovr_grounds_srating"] = result_df["motivo_principal_ovr"] == "SR"
result_df["ovr_grounds_exposure"] = result_df["motivo_principal_ovr"] == "exposure"
result_df["ovr_grounds_controversy"] = result_df["motivo_principal_ovr"] == "controversy"
result_df["ovr_grounds_committee"] = result_df["motivo_principal_ovr"] == "committee"
result_df["ovr_grounds_inheritance"] = result_df["motivo_principal_ovr"] == "inheritance"

ground_columns = [
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance"]

# for every column in in ground_columns, if value is not nan, then set the value to "x"
for col in ground_columns:
    result_df[col] = result_df[col].apply(lambda x: 'x' if x else np.nan)


In [30]:
result_df["ovr_grounds_srating"].value_counts(dropna=False)

ovr_grounds_srating
NaN    7180
x        21
Name: count, dtype: int64

In [31]:
for c in result_df.columns:
    print(c)

clarityid
permid
brs_id
issuer_name
parent_sub
parent_name_clarity
parent_clarityid
parent_name_brs
parent_brs_id
str_001_ovr
str_002_ovr
str_003_ovr
str_003b_ovr
str_004_ovr
str_005_ovr
str_006_ovr
art_8_ovr
cs_001_ovr
cs_003_ovr
cs_002_ovr
motivo_principal_ovr
motivo_secundario_ovr
company_inheriting
parent_company
str_001_df
str_002_df
str_003_df
str_003b_df
str_004_df
str_005_df
str_006_df
art_8_df
cs_001_df
cs_003_df
cs_002_df
str_001_ovr_active
str_002_ovr_active
str_003_ovr_active
str_003b_ovr_active
str_004_ovr_active
str_005_ovr_active
str_006_ovr_active
art_8_ovr_active
cs_001_ovr_active
cs_002_ovr_active
cs_003_ovr_active
ovr_grounds_srating
ovr_grounds_exposure
ovr_grounds_controversy
ovr_grounds_committee
ovr_grounds_inheritance


In [32]:
def load_result_to_dfs(result_df, basic_columns) -> list[pd.DataFrame]:

    dfs_list = []

    aux_col = [
    "ovr_grounds_srating",
    "ovr_grounds_exposure",
    "ovr_grounds_controversy",
    "ovr_grounds_committee",
    "ovr_grounds_inheritance",
    "motivo_principal_ovr",
    "motivo_secundario_ovr",
    "company_inheriting",
    "parent_company",
    ]

    strategy_columns = [
        "str_001_df", "str_002_df", "str_003_df", "str_003b_df",
        "str_004_df", "str_005_df", "str_006_df", "art_8_df",
        "cs_001_df", "cs_003_df", "cs_002_df",
    ]

    ovr_columns = [
        "str_001_ovr", "str_002_ovr", "str_003_ovr", "str_003b_ovr", 
        "str_004_ovr", "str_005_ovr", "str_006_ovr", "art_8_ovr", 
        "cs_001_ovr", "cs_003_ovr", "cs_002_ovr", 
    ]

    ovr_active_columns = [
        "str_001_ovr_active", "str_002_ovr_active", "str_003_ovr_active", "str_003b_ovr_active", 
        "str_004_ovr_active", "str_005_ovr_active", "str_006_ovr_active", "art_8_ovr_active", 
        "cs_001_ovr_active", "cs_002_ovr_active", "cs_003_ovr_active",     
    ]


    for strategy, ovr_col, active_col in zip(strategy_columns, ovr_columns, ovr_active_columns):
        result_df["ovr_target"] = strategy
        relevant_columns = basic_columns + ["ovr_target"] + [
            col for col in [strategy, ovr_col, active_col] if col in result_df.columns
        ] + aux_col
        if relevant_columns:
            dfs_list.append(result_df[relevant_columns])

    return dfs_list


In [33]:
# define basic columns before saving
basic_columns = [
    "clarityid",
    "permid",
    "brs_id",
    "issuer_name",
    "parent_sub",
    "parent_name_clarity",
    "parent_clarityid",
    "parent_name_brs",
    "parent_brs_id",
]
str_df_list = load_result_to_dfs(result_df, basic_columns=basic_columns)

In [34]:
# Loop through each strategy DataFrame in str_df_list to modify the names of the columns

def rename_column(col_name: str) -> str:
    """
    Check if the column name matches any of the patterns and
    return the appropriate replacement. If a column name matches
    more than one pattern, only the first match (in order) is used.
    """
    # Check for pattern ending with _ovr_active first.
    if re.search(r'.*_ovr_active$', col_name):
        return 'ovr_active'
    # Then check for pattern ending with _df.
    elif re.search(r'.*_df$', col_name):
        return 'df_value'
    # Then check for pattern ending with _ovr.
    elif re.search(r'.*_ovr$', col_name):
        return 'ovr_value'
    else:
        return col_name

def rename_dataframe_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a new DataFrame with the column names renamed according
    to the rename_column function.
    """
    new_columns = {col: rename_column(col) for col in df.columns}
    # Rename the DataFrame columns using the dictionary mapping.
    return df.rename(columns=new_columns)

# Apply the renaming function to each DataFrame in your list.
for i, df in enumerate(str_df_list):
    str_df_list[i] = rename_dataframe_columns(df)


In [35]:
# loop for str_df_list to filter non active overrides
str_df_list = [df[df['ovr_active'] == True] for df in str_df_list]

In [36]:
for df in str_df_list:
    df["ovr_target"] = df["ovr_target"].str.replace("_df", "")


In [37]:
# let's stack dataframes horizontally
final_df = pd.concat(str_df_list, axis=0, ignore_index=True)

In [38]:
ovr_target_columns_names={
        "str_001"   : "str_001_s",
        "str_002"   : "str_002_ec",
        "str_003"   : "str_003_ec",
        "str_003b"  : "str_003b_ec",
        "str_004"   : "str_004_asec",
        "str_005"   : "str_005_ec",
        "str_006"   : "str_006_sec",
        "cs_001"    : "cs_001_sec",
        "cs_003"    : "cs_003_sec",
        "cs_002"    : "cs_002_ec", 
        "art_8"     : "art_8_basicos"    
    }

In [39]:
# rename values of ovr_target column using ovr_target_columns_names
final_df["ovr_target"] = final_df["ovr_target"].map(ovr_target_columns_names)

for df in str_df_list:
    df["ovr_target"] = df["ovr_target"].map(ovr_target_columns_names)

In [40]:
final_df.head()

,clarityid,permid,brs_id,issuer_name,parent_sub,parent_name_clarity,parent_clarityid,parent_name_brs,parent_brs_id,ovr_target,...,ovr_active,ovr_grounds_srating,ovr_grounds_exposure,ovr_grounds_controversy,ovr_grounds_committee,ovr_grounds_inheritance,ovr_value,ovr_value,company_inheriting,parent_company
0,150114,5000014265,R65372,Agricultural Bank of China Ltd,P,-,150114,-,R65372,str_001_s,...,True,NaN,NaN,NaN,NaN,x,inheritance,-,False,-
1,66251,4295884955,R63005,Airbus SE,P,-,66251,Airbus SE,R63005,str_001_s,...,True,NaN,NaN,NaN,NaN,x,inheritance,-,False,-
2,262152,4297202523,R89955,Airbus Finance BV,S,Airbus SE,66251,Airbus SE,R63005,str_001_s,...,True,NaN,NaN,NaN,NaN,x,inheritance,-,True,Airbus SE
3,15657,4295869482,251525,Deutsche Bank AG,P,-,15657,DEUTSCHE BANK AG,251525,str_001_s,...,True,NaN,x,NaN,NaN,NaN,exposure,Controversia,False,-
4,171531900,5044066787,G88795,Al Shindagha Sukuk Ltd,S,Deutsche Bank AG,15657,DEUTSCHE BANK AG,251525,str_001_s,...,True,NaN,NaN,NaN,NaN,x,inheritance,-,True,Deutsche Bank AG


Now, let's save the results

In [41]:
# Define the mapping for sheet names based on the index in str_df_list.
sheet_names = [
    "str_001",  # index 0
    "str_002",  # index 1
    "str_003",  # index 2
    "str_003b", # index 3
    "str_004",  # index 4
    "str_005",  # index 5
    "str_006",  # index 6
    "art_8",    # index 7
    "cs_001",   # index 8
    "cs_003",   # index 9
    "cs_002",   # index 10
]
# define DATE
DATE = datetime.now().strftime("%Y%m%d")

# Specify the output Excel file name.
output_file = rf"output\{DATE}_override_analysis_subs.xlsx"

# Create an ExcelWriter object using the desired engine (e.g., xlsxwriter).
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # Write final_df to the first sheet named "summary".
    final_df.to_excel(writer, sheet_name="summary", index=False)
    
    # Loop over the list of DataFrames and write each to its corresponding sheet.
    for idx, df in enumerate(str_df_list):
        # Use the appropriate sheet name from the list.
        sheet_name = sheet_names[idx]
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [43]:
for df in str_df_list:
    print(f"The strategy {df.ovr_target.unique()[0]} has: {len(df.issuer_name.unique())} issuers and {len(df)} overrides")

The strategy str_001_s has: 580 issuers and 581 overrides
The strategy str_002_ec has: 724 issuers and 725 overrides
The strategy str_003_ec has: 532 issuers and 533 overrides
The strategy str_003b_ec has: 298 issuers and 299 overrides
The strategy str_004_asec has: 722 issuers and 723 overrides
The strategy str_005_ec has: 1288 issuers and 1289 overrides
The strategy str_006_sec has: 837 issuers and 838 overrides
The strategy art_8_basicos has: 391 issuers and 391 overrides
The strategy cs_001_sec has: 593 issuers and 593 overrides
The strategy cs_003_sec has: 587 issuers and 589 overrides
The strategy cs_002_ec has: 1108 issuers and 1110 overrides


In [42]:
len(final_df.issuer_name.unique()) / 3

778.6666666666666

In [45]:
len(final_df.issuer_name.unique())

2336

In [46]:
len(final_df.parent_brs_id.unique())

219

In [48]:
len(final_df.brs_id.unique()) / 3


556.0

In [44]:
# print names that appear more than once in the df from str_df_list
for df in str_df_list:
    print(df.ovr_target.unique()[0])
    print(df.issuer_name.value_counts()[df.issuer_name.value_counts() > 1])
    print("\n")

str_001_s
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


str_002_ec
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


str_003_ec
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


str_003b_ec
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


str_004_asec
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


str_005_ec
issuer_name
Macquarie Bank Ltd    2
Name: count, dtype: int64


str_006_sec
issuer_name
Unicredit SPA    2
Name: count, dtype: int64


art_8_basicos
Series([], Name: count, dtype: int64)


cs_001_sec
Series([], Name: count, dtype: int64)


cs_003_sec
issuer_name
Unicredit SPA      2
CSOB Leasing as    2
Name: count, dtype: int64


cs_002_ec
issuer_name
Unicredit SPA      2
CSOB Leasing as    2
Name: count, dtype: int64


